In [170]:
from lasscraping.scraping import time_random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, StaleElementReferenceException

import undetected_chromedriver as uc 

import pandas as pd
from tqdm import tqdm
from IPython.display import HTML, display
from bs4 import BeautifulSoup
import re 
import os


In [94]:
driver = uc.Chrome()

In [95]:
driver.implicitly_wait(15)
wait = WebDriverWait(driver, 10)

In [96]:
driver.get('https://eproc1g.tjsc.jus.br/eproc/externo_controlador.php?acao=jurisprudencia@jurisprudencia/pesquisar')

### FUNCOES

In [6]:
blocos = driver.find_element(By.CSS_SELECTOR, '#bodyResultados').get_property('children')

In [7]:
blocos[0]

<undetected_chromedriver.webelement.WebElement (session="925c50c454400443b2b53c67dd15fdfa", element="f.8096122AB224B961DF326F9699FE3122.d.A79CA9D317246B850C576421E832C256.e.349")>

In [10]:
try:
    processo_label = blocos[0].find_element(By.XPATH, './/*[text()="PROCESSO"]')
    numero_processo_value = processo_label.get_property('nextElementSibling').find_element(By.CSS_SELECTOR, 'a')
except NoSuchElementException as e:
    print('Numero Processo sem registro')

In [35]:
try:
    data_julgamento_label = bloco[0].find_element(By.XPATH, './/*[text()="DATA DO JULGAMENTO"]')
    data_julgamento_value = data_julgamento_label.get_property('nextElementSibling')
except NoSuchElementException as e:
    print('Data julgamento sem registro')

In [37]:
try:
    data_publi_label = bloco[0].find_element(By.XPATH, './/*[text()="DATA DA PUBLICAÇÃO"]')
    data_publi_value = data_publi_label.get_property('nextElementSibling')
except NoSuchElementException as e:
    print('Data publicação sem registro')

In [62]:
try:
    relato_label = bloco[0].find_element(By.XPATH, './/*[text()="RELATOR"]')
    relator_value = relato_label.get_property('nextElementSibling')
except NoSuchElementException as e:
    print('Relator sem registro')

In [66]:
try:
    revisor_label = bloco[0].find_element(By.XPATH, './/*[text()="REVISOR"]')
    revisor_value = revisor_label.get_property('nextElementSibling')
except NoSuchElementException as e:
    print('Revisor sem registro')

Revisor sem registro


In [67]:
try:
    decisao_label = bloco[0].find_element(By.XPATH, './/*[text()="DECISÃO"]')
    decisao_value = decisao_label.get_property('nextElementSibling')
except NoSuchElementException as e:
    print('Decisao sem registro')

In [69]:
try:
    ementa_label = bloco[0].find_element(By.XPATH, './/*[text()="EMENTA"]')
    ementa_value = ementa_label.get_property('nextElementSibling')
except NoSuchElementException as e:
    print('Decisao sem registro')

In [72]:
from IPython.display import HTML, display
from bs4 import BeautifulSoup
import re 

In [16]:
blocos = driver.find_elements(By.CSS_SELECTOR, '#bodyResultados > div.card > div:nth-child(2) > div')

In [164]:
def scraping_bloco_bs4(bloco):
    
    html = bloco.get_property('innerHTML')
    soup = BeautifulSoup(html, 'html.parser' )

    data = {}

    try:
        numero_processo_value = soup.find('div', string='PROCESSO').find_next_sibling().a.get_text().strip()
        data['processo'] = numero_processo_value
    except AttributeError as e:
        data['processo'] = None

    try:
        data_julgamento_value = soup.find('div', string='DATA DO JULGAMENTO').find_next_sibling().get_text().strip()
        data['data_julgamento'] = data_julgamento_value
    except AttributeError as e:
        data['data_julgamento'] = None

    try:
        data_publi_value = soup.find('div', string='DATA DA PUBLICAÇÃO').find_next_sibling().get_text().strip()
        data['data_publicacao'] = data_publi_value
    except AttributeError as e:
        data['data_publicacao'] = None

    try:
        relator_value = soup.find('div', string=re.compile(r'RELATOR(A)?')).find_next_sibling().get_text().strip()
        data['relator'] = relator_value
    except AttributeError as e:
        data['relator'] = None

    try:
        revisor_value = soup.find('div', string=re.compile(r'REVISOR(A)?')).find_next_sibling().get_text().strip()
        data['revisor'] = revisor_value
    except AttributeError as e:
        data['revisor'] = None

    try:
        decisao_value = soup.find('div', string='DECISÃO').find_next_sibling().get_text().strip()
        data['decisao'] = decisao_value
    except AttributeError as e:
        data['decisao'] = None

    try:
        ementa_value = soup.find('div', string='EMENTA').find_next_sibling().get_text().strip()
        data['ementa'] = ementa_value
    except AttributeError as e:
        data['ementa'] = None
    


    relatorio_link = f'https://eproc2g.tjsc.jus.br/eproc/{bloco.find_element(By.CSS_SELECTOR, 'h4 > div > span > a[title="Exibir inteiro teor do documento"]').get_attribute('data-link')}'
    driver.switch_to.new_window('tab')

    driver.switch_to.window(driver.window_handles[-1])

    driver.get(relatorio_link)
    time_random(0.5,0.5)

    texto = driver.find_element(By.TAG_NAME, 'html').text
    data['relatorio'] = texto
    data['link_relatorio'] = driver.current_url

    driver.close()
    driver.switch_to.window(driver.window_handles[0])

    try:
        link_processo = soup.find('div', string='PROCESSO').find_next_sibling().a['href'].strip()
        data['link_processo'] = link_processo
    except AttributeError as e:
        data['link_processo'] = None

    return data

In [125]:
def scraping_blocos(bloco):
    """Extrai campos de um bloco de resultados e retorna um dict."""
    data = {}
    try:
        processo_label = bloco.find_element(By.XPATH, './/*[text()="PROCESSO"]')
        numero_processo_value = processo_label.get_property('nextElementSibling').find_element(By.CSS_SELECTOR, 'a').text
        data['processo'] = numero_processo_value
    except NoSuchElementException:
        data['processo'] = None
    try:
        data_julgamento_label = bloco.find_element(By.XPATH, './/*[text()="DATA DO JULGAMENTO"]')
        data_julgamento_value = data_julgamento_label.get_property('nextElementSibling').text
        data['data_julgamento'] = data_julgamento_value
    except NoSuchElementException:
        data['data_julgamento'] = None
    try:
        data_publi_label = bloco.find_element(By.XPATH, './/*[text()="DATA DA PUBLICAÇÃO"]')
        data_publi_value = data_publi_label.get_property('nextElementSibling').text
        data['data_publicacao'] = data_publi_value
    except NoSuchElementException:
        data['data_publicacao'] = None
    try:
        relator_label = bloco.find_element(By.XPATH, './/*[text()="RELATOR"]')
        relator_value = relator_label.get_property('nextElementSibling').text
        data['relator'] = relator_value
    except NoSuchElementException:
        data['relator'] = None
    try:
        revisor_label = bloco.find_element(By.XPATH, './/*[text()="REVISOR"]')
        revisor_value = revisor_label.get_property('nextElementSibling').text
        data['revisor'] = revisor_value
    except NoSuchElementException:
        data['revisor'] = None
    try:
        decisao_label = bloco.find_element(By.XPATH, './/*[text()="DECISÃO"]')
        decisao_value = decisao_label.get_property('nextElementSibling').text
        data['decisao'] = decisao_value
    except NoSuchElementException:
        data['decisao'] = None
    try:
        ementa_label = bloco.find_element(By.XPATH, './/*[text()="EMENTA"]')
        ementa_value = ementa_label.get_property('nextElementSibling').text
        data['ementa'] = ementa_value
    except NoSuchElementException:
        data['ementa'] = None
    return data

### COD PRINCIPAL

In [171]:
data = []

CAMINHO_ARQUIVO = os.path.join('data', 'maus-tratos_atualizado.csv')


with tqdm(unit='página', desc='Coletando dados', total=None) as pbar:
    while True:
        
        blocos = driver.find_elements(By.CSS_SELECTOR, '#bodyResultados > div.card')
        for bloco in blocos:
            d = scraping_bloco_bs4(bloco)
            data.append(d)

        pbar.update(1)
        proximo = driver.find_element(By.CSS_SELECTOR, '#headerResultados > div > div > div.justify-content-md-end > div.mr-3 > i[title="Ir para próxima página"]')
        if 'iconeDesativado' in proximo.get_attribute('class'):
            break
        else:
            proximo.click()
            wait.until(EC.staleness_of(blocos[0]))
            scraping.time_random(5.3, 10)

df = pd.DataFrame(data)
df.to_csv(caminho, index=False)

Coletando dados: 14página [21:34, 92.48s/página]


NameError: name 'caminho' is not defined

In [172]:
df.to_csv(CAMINHO_ARQUIVO, index=False)

In [180]:
from bs4 import BeautifulSoup, Tag
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import pandas as pd
import time
import os

# --- CONFIGURAÇÕES ---
CAMINHO_ARQUIVO = os.path.join('data', 'maus-tratos.csv')

def _limpar_texto(texto: str) -> str:
    """Remove espaços extras e quebras de linha."""
    if texto:
        return " ".join(texto.split())
    return None

def scraping_blocos_bs4(bloco_bs4: Tag) -> dict:
    """
    Extrai dados de um bloco usando uma estratégia de varredura de labels.
    Isso é mais seguro porque pega 'RELATOR' ou 'RELATORA' automaticamente.
    """
    data = {
        'processo': None,
        'data_julgamento': None,
        'data_publicacao': None,
        'relator': None,
        'revisor': None,
        'decisao': None,
        'ementa': None
    }
    
    # Encontra todos os rótulos (títulos dos campos) dentro deste bloco
    labels = bloco_bs4.find_all('div', class_='resLabel')
    
    for label in labels:
        texto_label = label.get_text(strip=True).upper()
        
        # Pega o valor vizinho (que contém o dado real)
        valor_div = label.find_next_sibling('div', class_='resValue')
        if not valor_div:
            continue
            
        texto_valor = valor_div.get_text(strip=True)
        
        # --- Mapeamento dos Campos ---
        
        if "PROCESSO" in texto_label:
            # O processo geralmente está dentro de um link <a> com classe 'numero-processo'
            link = valor_div.select_one('a.numero-processo')
            if link:
                data['processo'] = _limpar_texto(link.get_text())
            else:
                data['processo'] = _limpar_texto(texto_valor)
                
        elif "DATA DO JULGAMENTO" in texto_label:
            data['data_julgamento'] = _limpar_texto(texto_valor)
            
        elif "DATA DA PUBLICAÇÃO" in texto_label:
            data['data_publicacao'] = _limpar_texto(texto_valor)
            
        # "RELATOR" ou "RELATORA" (cobre os dois casos)
        elif "RELATOR" in texto_label: 
            data['relator'] = _limpar_texto(texto_valor)
            
        # "REVISOR" ou "REVISORA"
        elif "REVISOR" in texto_label:
            data['revisor'] = _limpar_texto(texto_valor)
            
        elif "DECISÃO" in texto_label:
            data['decisao'] = _limpar_texto(texto_valor)
            
        elif "EMENTA" in texto_label:
            data['ementa'] = _limpar_texto(texto_valor)

    return data

# --- LOOP PRINCIPAL ---

dados_coletados = []

# Certifique-se de que a pasta 'data' existe
os.makedirs('data', exist_ok=True)

# Loop infinito com barra de progresso manual
with tqdm(desc='Páginas Coletadas', unit='pág') as pbar:
    while True:
        try:
            # 1. Pega o HTML atual
            html_completo = driver.page_source
            soup = BeautifulSoup(html_completo, 'html.parser')
            
            # 2. Seleciona os blocos de resultado
            # Baseado no seu arquivo: <div class="card mb-3 resultadoItem">
            blocos = soup.find_all('div', class_=['card', 'mb-3', 'resultadoItem'])
            
            if not blocos:
                print("Nenhum bloco encontrado nesta página (ou fim da busca).")
                break

            # 3. Processa os blocos com BS4 (Rápido)
            for bloco in blocos:
                item = scraping_blocos_bs4(bloco)
                
                dados_coletados.append(item)
            len(dados_coletados)
            
            # Atualiza a barra de progresso e salva parcial (segurança)
            pbar.update(1)
            # Opcional: Salvar a cada X páginas para não perder tudo se der erro
            if len(dados_coletados) % 100 == 0:
                 pd.DataFrame(dados_coletados).to_csv(caminho_arquivo, index=False)

            # 4. Paginação (via Selenium)
            # Procura o botão "Próxima página"
            botao_proximo = driver.find_element(By.CSS_SELECTOR, 'i[title="Ir para próxima página"]')
            botao_pai = botao_proximo.find_element(By.XPATH, '..') # Pega o elemento pai (div ou a)

            # Verifica se o botão está desativado (classe 'iconeDesativado' no <i> ou no pai)
            classe_icone = botao_proximo.get_attribute('class')
            classe_pai = botao_pai.get_attribute('class')
            
            if 'iconeDesativado' in classe_icone or (classe_pai and 'disabled' in classe_pai):
                print("Fim da paginação alcançado.")
                break
            
            # Clica para avançar
            botao_proximo.click()
            
            # Espera inteligente: aguarda o primeiro bloco ficar "velho" (stale)
            # Isso garante que a página nova carregou antes de ler o HTML de novo
            wait = WebDriverWait(driver, 15)
            # Precisamos pegar um elemento via Selenium para o 'staleness_of'
            elemento_antigo = driver.find_element(By.CSS_SELECTOR, '.resultadoItem')
            wait.until(EC.staleness_of(elemento_antigo))
            
            # Pequeno delay aleatório para evitar bloqueio (WAF)
            scraping.time_random(5.0, 7.0) 

        except Exception as e:
            print(f"Erro no loop: {e}")
            # Tenta salvar o que já pegou antes de sair
            break

# --- SALVAMENTO FINAL ---
if dados_coletados:
    df = pd.DataFrame(dados_coletados)
    df.to_csv(CAMINHO_ARQUIVO, index=False)
    print(f"Sucesso! {len(df)} registros salvos em {CAMINHO_ARQUIVO}")
else:
    print("Nenhum dado foi coletado.")

Páginas Coletadas: 7pág [00:50,  7.16s/pág]

Fim da paginação alcançado.
Sucesso! 731 registros salvos em data\maus-tratos.csv


In [185]:
df_limpo = df.dropna(subset=['processo'])

In [187]:
df_limpo.to_csv(CAMINHO_ARQUIVO)

## ETL

In [173]:
df

,processo,data_julgamento,data_publicacao,relator,revisor,decisao,ementa,relatorio,link_relatorio,link_processo
0,5001637-86.2024.8.24.0139/TJSC,09/12/2025,04/12/2025,CINTHIA BEATRIZ DA SILVA BITTENCOURT SCHAEFER,LUIZ CESAR SCHWEITZER,Vistos e relatados estes autos em que são part...,APELAÇÃO CRIMINAL. CRIME CONTRA O MEIO AMBIENT...,ESTADO DE SANTA CATARINA\nTRIBUNAL DE JUSTIÇA\...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...
1,5000591-28.2024.8.24.0021/TJSC,09/12/2025,04/12/2025,LUIZ NERI OLIVEIRA DE SOUZA,VOLNEI CELSO TOMAZINI,Vistos e relatados estes autos em que são part...,APELAÇÃO CRIMINAL. MAUS TRATOS CONTRA ANIMAIS ...,ESTADO DE SANTA CATARINA\nTRIBUNAL DE JUSTIÇA\...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...
2,5005896-60.2023.8.24.0010/TJSC,04/12/2025,03/12/2025,ARIOVALDO ROGÉRIO RIBEIRO DA SILVA,ANA LIA MOURA LISBOA CARNEIRO,Vistos e relatados estes autos em que são part...,DIREITO PENAL E PROCESSUAL PENAL. APELAÇÃO CRI...,ESTADO DE SANTA CATARINA\nTRIBUNAL DE JUSTIÇA\...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...
3,5011414-15.2024.8.24.0004/TJSC,04/12/2025,03/12/2025,PAULO ROBERTO SARTORATO,CARLOS ALBERTO CIVINSKI,Vistos e relatados estes autos em que são part...,APELAÇÃO CRIMINAL. CRIME DE MAUS-TRATOS DE CÃE...,ESTADO DE SANTA CATARINA\nTRIBUNAL DE JUSTIÇA\...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...
4,5003298-46.2023.8.24.0039/TJSC,02/12/2025,26/11/2025,NORIVAL ACÁCIO ENGEL,None,Vistos e relatados estes autos em que são part...,APELAÇÃO CRIMINAL. MAUS-TRATOS A ANIMAIS DOMÉS...,ESTADO DE SANTA CATARINA\nTRIBUNAL DE JUSTIÇA\...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...
...,...,...,...,...,...,...,...,...,...,...
1268,4008316-95.2017.8.24.0000/TJSC,,19/09/2017,JOAO HENRIQUE BLASI,None,None,None,ESTADO DE SANTA CATARINA\n TRIBUNAL DE...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,externo_controlador.php?acao=consultaArquivado...
1269,4026473-82.2018.8.24.0000/TJSC,,03/10/2018,SÉRGIO RIZELO,None,None,None,Habeas Corpus n. 4026473-82.2018.8.24.0000\n\n...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,externo_controlador.php?acao=consultaArquivado...
1270,4032299-55.2019.8.24.0000/TJSC,,05/12/2019,PAULO HENRIQUE MORITZ MARTINS DA SILVA,None,None,None,Agravo de Instrumento n. 4032299-55.2019.8.24....,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,externo_controlador.php?acao=consultaArquivado...
1271,0006868-40.2013.8.24.0019/TJSC,,19/05/2020,MONTEIRO ROCHA,None,None,None,Quinta Câmara de Direito Comercial\nDesembarga...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,externo_controlador.php?acao=consultaArquivado...


In [195]:
import pandas as pd
import re

def identificar_animal_completo(texto):
    """
    Classifica o animal com base em palavras-chave no texto do relatório.
    Ordem de prioridade: Cão/Gato > Equinos > Aves > Suínos > Bovinos > Outros.
    """
    if not isinstance(texto, str):
        return "Não identificado"
    
    # Normaliza para minúsculo para simplificar os regex
    texto = texto.lower()
    
    # --- Padrões de Regex (Grupos de Animais) ---
    
    # 1. Cães e Gatos (Pet)
    # Ex: cachorro, cadela, cão, cães, pitbull, pinscher, gato, felino
    rgx_cachorro = r'cachorr[oa]s?|c[ãa]os?|c[ãa]es|cadelas?|canin[oa]s?|pit\s?bull|rottweiler'
    rgx_gato     = r'gat[oa]s?|felin[oa]s?'
    
    # 2. Aves (muito comum em casos de Rinha ou Animais Silvestres em gaiolas)
    # Ex: galo, galinha, ave, pássaro, passeriforme, curió, trinca-ferro, canário, rinha
    rgx_ave      = r'gal[oa]s?|aves?|p[áa]ssaros?|passeriformes?|rinha|gaiolas?|can[áa]ri[oa]s?|curi[óo]|trinca[- ]?ferro'
    
    # 3. Equinos (muito comum em casos de tração/carroça)
    # Ex: cavalo, égua, equino, potro, burro, mula, tração animal
    rgx_equino   = r'caval[oa]s?|[ée]guas?|equin[oa]s?|potr[oa]s?|burr[oa]s?'
    
    # 4. Suínos (Seu exemplo anterior) e Bovinos (Gado/Fazenda)
    rgx_suino    = r'su[íi]n[oa]s?|porc[oa]s?|leit[ãa]o|leit[õo]es|chiqueiro'
    rgx_bovino   = r'bovin[oa]s?|gad[oa]|vacas?|bois?|bezerros?'

    # --- Verificação Sequencial ---
    
    # DICA: A ordem dos 'if' define a prioridade se o texto falar de vários bichos.
    
    tem_cachorro = re.search(rgx_cachorro, texto)
    tem_gato     = re.search(rgx_gato, texto)
    
    if tem_cachorro and tem_gato:
        return "Misto (Cão e Gato)"
    elif tem_cachorro:
        return "Cachorro"
    elif tem_gato:
        return "Gato"
    
    # Se não for pet, verifica os outros grupos
    if re.search(rgx_equino, texto):
        return "Equino (Cavalo/Burro)"
    
    if re.search(rgx_ave, texto):
        return "Ave (Galo/Pássaro)"
        
    if re.search(rgx_suino, texto):
        return "Suíno (Porco)"
        
    if re.search(rgx_bovino, texto):
        return "Bovino (Gado/Vaca)"
        
    # Se chegou aqui, é outro bicho ou não foi citado explicitamente
    return "Outro/Não identificado"

# APLICANDO
# Supondo que seu DataFrame se chame 'df' e a coluna de texto seja 'relatorio'
df['classificacao_animal'] = df['relatorio'].apply(identificar_animal_completo)

# Visualizando a contagem final
print(df['classificacao_animal'].value_counts())

classificacao_animal
Ave (Galo/Pássaro)        408
Cachorro                  294
Misto (Cão e Gato)        238
Outro/Não identificado    148
Gato                      102
Bovino (Gado/Vaca)         43
Equino (Cavalo/Burro)      37
Suíno (Porco)               3
Name: count, dtype: int64


In [211]:
import pandas as pd
import re

def extrair_comarca(texto):
    """
    Busca o padrão 'Comarca de [NOME],' e retorna apenas o NOME.
    Ignora maiúsculas/minúsculas na busca.
    """
    if not isinstance(texto, str):
        return None
    
    # --- Explicação do Regex ---
    # comarca\s+de\s+  -> Procura "comarca de" (com espaços variados no meio)
    # ([^,]+)          -> CAPTURA tudo que vier depois, DESDE QUE não seja vírgula
    # ,                -> Para de capturar quando encontrar a vírgula
    padrao = r'comarca\s+de\s+([^,]+),'
    
    # re.IGNORECASE faz com que 'Comarca', 'comarca' ou 'COMARCA' sejam aceitos
    match = re.search(padrao, texto, re.IGNORECASE)
    
    if match:
        # group(1) pega apenas o conteúdo que estava dentro dos parênteses ([^,]+)
        return match.group(1).strip() # .strip() remove espaços em branco sobrando nas pontas
    
    return None

# APLICANDO NO SEU DATAFRAME
# Cria uma nova coluna chamada 'comarca'
df['comarca'] = df['relatorio'].apply(extrair_comarca)

# Visualizando o resultado
print(df[['processo', 'comarca']].head())

# Para verificar quantas comarcas diferentes você capturou:
# print(df['comarca'].value_counts())

                         processo  \
0  5001637-86.2024.8.24.0139/TJSC   
1  5000591-28.2024.8.24.0021/TJSC   
2  5005896-60.2023.8.24.0010/TJSC   
3  5011414-15.2024.8.24.0004/TJSC   
4  5003298-46.2023.8.24.0039/TJSC   

                                             comarca  
0                                               None  
1                                         Cunha Porã  
2  Braço do Norte que condenou o acusado V. M. T....  
3                                          Araranguá  
4                                              Lages  


In [212]:
import unicodedata

# 1. LISTA DE GABARITO (Essencial para o Python saber o que procurar)
municipios_sc = [
    "Abelardo Luz", "Agrolândia", "Agronômica", "Água Doce", "Águas de Chapecó", "Águas Frias", "Águas Mornas",
    "Alfredo Wagner", "Alto Bela Vista", "Anchieta", "Angelina", "Anita Garibaldi", "Anitápolis", "Antônio Carlos",
    "Apiúna", "Arabutã", "Araquari", "Araranguá", "Armazém", "Arroio Trinta", "Arvoredo", "Ascurra", "Atalanta",
    "Aurora", "Balneário Arroio do Silva", "Balneário Barra do Sul", "Balneário Camboriú", "Balneário Gaivota",
    "Balneário Piçarras", "Balneário Rincão", "Bandeirante", "Barra Bonita", "Barra Velha", "Bela Vista do Toldo",
    "Belmonte", "Benedito Novo", "Biguaçu", "Blumenau", "Bocaina do Sul", "Bom Jardim da Serra", "Bom Jesus",
    "Bom Jesus do Oeste", "Bom Retiro", "Bombinhas", "Botuverá", "Braço do Norte", "Braço do Trombudo", "Brunópolis",
    "Brusque", "Caçador", "Caibi", "Calmon", "Camboriú", "Campo Alegre", "Campo Belo do Sul", "Campo Erê",
    "Campos Novos", "Canelinha", "Canoinhas", "Capão Alto", "Capinzal", "Capivari de Baixo", "Catanduvas",
    "Caxambu do Sul", "Celso Ramos", "Cerro Negro", "Chapadão do Lageado", "Chapecó", "Cocal do Sul", "Concórdia",
    "Cordilheira Alta", "Coronel Freitas", "Coronel Martins", "Correia Pinto", "Corupá", "Criciúma", "Cunha Porã",
    "Cunhataí", "Curitibanos", "Descanso", "Dionísio Cerqueira", "Dona Emma", "Doutor Pedrinho", "Entre Rios",
    "Ermo", "Erval Velho", "Faxinal dos Guedes", "Flor do Sertão", "Florianópolis", "Formosa do Sul", "Forquilhinha",
    "Fraiburgo", "Frei Rogério", "Galvão", "Garopaba", "Garuva", "Gaspar", "Governador Celso Ramos", "Grão-Pará",
    "Gravatal", "Guabiruba", "Guaraciaba", "Guaramirim", "Guarujá do Sul", "Guatambu", "Herval d'Oeste", "Ibiam",
    "Ibicaré", "Ibirama", "Içara", "Ilhota", "Imaruí", "Imbituba", "Imbuia", "Indaial", "Iomerê", "Ipira", "Iporã do Oeste",
    "Ipuaçu", "Ipumirim", "Iraceminha", "Irani", "Irati", "Irineópolis", "Itá", "Itaiópolis", "Itajaí", "Itapema",
    "Itapiranga", "Itapoá", "Ituporanga", "Jaborá", "Jacinto Machado", "Jaguaruna", "Jaraguá do Sul", "Jardinópolis",
    "Joaçaba", "Joinville", "José Boiteux", "Jupiá", "Lacerdópolis", "Lages", "Laguna", "Lajeado Grande", "Laurentino",
    "Lauro Müller", "Lebon Régis", "Leoberto Leal", "Lindóia do Sul", "Lontras", "Luiz Alves", "Luzerna", "Macieira",
    "Mafra", "Major Gercino", "Major Vieira", "Maracajá", "Maravilha", "Marema", "Massaranduba", "Matos Costa",
    "Meleiro", "Mirim Doce", "Modelo", "Mondaí", "Monte Carlo", "Monte Castelo", "Morro da Fumaça", "Morro Grande",
    "Navegantes", "Nova Erechim", "Nova Itaberaba", "Nova Trento", "Nova Veneza", "Novo Horizonte", "Orleans",
    "Otacílio Costa", "Ouro", "Ouro Verde", "Paial", "Painel", "Palhoça", "Palma Sola", "Palmeira", "Palmitos",
    "Papanduva", "Paraíso", "Passo de Torres", "Passos Maia", "Paulo Lopes", "Pedras Grandes", "Penha", "Peritiba",
    "Pescaria Brava", "Petrolândia", "Pinhalzinho", "Pinheiro Preto", "Piratuba", "Planalto Alegre", "Pomerode",
    "Ponte Alta", "Ponte Alta do Norte", "Ponte Serrada", "Porto Belo", "Porto União", "Pouso Redondo", "Praia Grande",
    "Presidente Castello Branco", "Presidente Getúlio", "Presidente Nereu", "Princesa", "Quilombo", "Rancho Queimado",
    "Rio das Antas", "Rio do Campo", "Rio do Oeste", "Rio do Sul", "Rio dos Cedros", "Rio Fortuna", "Rio Negrinho",
    "Rio Rufino", "Riqueza", "Rodeio", "Romelândia", "Salete", "Saltinho", "Salto Veloso", "Sangão", "Santa Cecília",
    "Santa Helena", "Santa Rosa de Lima", "Santa Rosa do Sul", "Santa Terezinha", "Santa Terezinha do Progresso",
    "Santiago do Sul", "Santo Amaro da Imperatriz", "São Bento do Sul", "São Bernardino", "São Bonifácio",
    "São Carlos", "São Cristóvão do Sul", "São Domingos", "São Francisco do Sul", "São João Batista",
    "São João do Itaperiú", "São João do Oeste", "São João do Sul", "São Joaquim", "São José", "São José do Cedro",
    "São José do Cerrito", "São Lourenço do Oeste", "São Ludgero", "São Martinho", "São Miguel da Boa Vista",
    "São Miguel do Oeste", "São Pedro de Alcântara", "Saudades", "Schroeder", "Seara", "Serra Alta", "Siderópolis",
    "Sombrio", "Sul Brasil", "Taió", "Tangará", "Tigrunhos", "Tijucas", "Timbé do Sul", "Timbó", "Timbó Grande",
    "Três Barras", "Treviso", "Treze de Maio", "Treze Tílias", "Trombudo Central", "Tubarão", "Tunápolis", "Turvo",
    "União do Oeste", "Urubici", "Urupema", "Urussanga", "Vargeão", "Vargem", "Vargem Bonita", "Vidal Ramos",
    "Videira", "Vitor Meireles", "Witmarsum", "Xanxerê", "Xavantina", "Xaxim", "Zortéa"
]

# 2. FUNÇÃO DE LIMPEZA
def limpar_municipio(texto):
    if not isinstance(texto, str):
        return None
    
    # Normaliza o texto da linha (tira acento, põe minúscula)
    texto_norm = texto.replace("’", "'").replace("d’", "d'")
    texto_norm = unicodedata.normalize('NFKD', texto_norm).encode('ASCII', 'ignore').decode('utf-8').lower().strip()
    
    # Ordena cidades da maior para a menor para evitar erro (ex: achar "São Miguel" dentro de "São Miguel do Oeste")
    cidades_ordenadas = sorted(municipios_sc, key=len, reverse=True)
    
    for cidade in cidades_ordenadas:
        cidade_norm = unicodedata.normalize('NFKD', cidade).encode('ASCII', 'ignore').decode('utf-8').lower()
        if cidade_norm in texto_norm:
            return cidade # Retorna o nome bonitinho da lista oficial
            
    return None # Se não achar nada

# 3. APLICAR NO SEU DATAFRAME EXISTENTE
# Troque 'nome_da_sua_coluna' pelo nome real da coluna no seu df
# O resultado vai sobrescrever a coluna suja. Se quiser criar uma nova, mude o nome da esquerda.

df['comarca'] = df['comarca'].apply(limpar_municipio)

# Verifique o resultado
print(df['comarca'].unique())

[None 'Cunha Porã' 'Braço do Norte' 'Araranguá' 'Lages' 'Joinville'
 'Caçador' 'Quilombo' 'Porto União' 'Criciúma' 'Biguaçu' 'Xaxim' 'Gaspar'
 'Xanxerê' 'Dionísio Cerqueira' 'Blumenau' 'Presidente Getúlio'
 'Rio do Campo' 'Mafra' 'Pomerode' 'Canoinhas' 'São Miguel do Oeste'
 'Turvo' 'Armazém' 'Rio Negrinho' 'Forquilhinha' 'Jaguaruna' 'Chapecó'
 'São Domingos' 'Santo Amaro da Imperatriz' 'Bom Retiro' 'Itajaí'
 'Catanduvas' 'Tubarão' 'São Joaquim' 'Pinhalzinho' 'Balneário Piçarras'
 'São Francisco do Sul' 'Sombrio' 'Garuva' 'Palhoça' 'Balneário Camboriú'
 'Jaraguá do Sul' 'Barra Velha' 'Concórdia' 'Itapoá' 'Campos Novos'
 'Joaçaba' 'Abelardo Luz' 'Rio do Oeste' 'Itá' 'Videira' 'Brusque'
 'Correia Pinto' 'Itapiranga' 'São Carlos' 'Ponte Serrada' 'Capinzal'
 'Campo Belo do Sul' 'Rio do Sul' 'Penha' 'Laguna' 'Indaial' 'Itapema'
 'Anchieta' 'Fraiburgo' 'Ibirama' 'Ipumirim' 'Camboriú' 'Imbituba' 'Seara'
 'Navegantes' 'Lauro Müller' 'Garopaba' 'Curitibanos' 'Coronel Freitas'
 'Maravilha' 'São 

In [213]:
df

,processo,data_julgamento,data_publicacao,relator,revisor,decisao,ementa,relatorio,link_relatorio,link_processo,animal_identificado,classificacao_animal,comarca
0,5001637-86.2024.8.24.0139/TJSC,09/12/2025,04/12/2025,CINTHIA BEATRIZ DA SILVA BITTENCOURT SCHAEFER,LUIZ CESAR SCHWEITZER,Vistos e relatados estes autos em que são part...,APELAÇÃO CRIMINAL. CRIME CONTRA O MEIO AMBIENT...,ESTADO DE SANTA CATARINA\nTRIBUNAL DE JUSTIÇA\...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,Ambos (Cachorro e Gato),Misto (Cão e Gato),None
1,5000591-28.2024.8.24.0021/TJSC,09/12/2025,04/12/2025,LUIZ NERI OLIVEIRA DE SOUZA,VOLNEI CELSO TOMAZINI,Vistos e relatados estes autos em que são part...,APELAÇÃO CRIMINAL. MAUS TRATOS CONTRA ANIMAIS ...,ESTADO DE SANTA CATARINA\nTRIBUNAL DE JUSTIÇA\...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,Ambos (Cachorro e Gato),Misto (Cão e Gato),Cunha Porã
2,5005896-60.2023.8.24.0010/TJSC,04/12/2025,03/12/2025,ARIOVALDO ROGÉRIO RIBEIRO DA SILVA,ANA LIA MOURA LISBOA CARNEIRO,Vistos e relatados estes autos em que são part...,DIREITO PENAL E PROCESSUAL PENAL. APELAÇÃO CRI...,ESTADO DE SANTA CATARINA\nTRIBUNAL DE JUSTIÇA\...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,Gato,Gato,Braço do Norte
3,5011414-15.2024.8.24.0004/TJSC,04/12/2025,03/12/2025,PAULO ROBERTO SARTORATO,CARLOS ALBERTO CIVINSKI,Vistos e relatados estes autos em que são part...,APELAÇÃO CRIMINAL. CRIME DE MAUS-TRATOS DE CÃE...,ESTADO DE SANTA CATARINA\nTRIBUNAL DE JUSTIÇA\...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,Ambos (Cachorro e Gato),Misto (Cão e Gato),Araranguá
4,5003298-46.2023.8.24.0039/TJSC,02/12/2025,26/11/2025,NORIVAL ACÁCIO ENGEL,NaN,Vistos e relatados estes autos em que são part...,APELAÇÃO CRIMINAL. MAUS-TRATOS A ANIMAIS DOMÉS...,ESTADO DE SANTA CATARINA\nTRIBUNAL DE JUSTIÇA\...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,Cachorro,Cachorro,Lages
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,4008316-95.2017.8.24.0000/TJSC,NaN,19/09/2017,JOAO HENRIQUE BLASI,NaN,NaN,NaN,ESTADO DE SANTA CATARINA\n TRIBUNAL DE...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,externo_controlador.php?acao=consultaArquivado...,Outro/Não identificado,Ave (Galo/Pássaro),None
1269,4026473-82.2018.8.24.0000/TJSC,NaN,03/10/2018,SÉRGIO RIZELO,NaN,NaN,NaN,Habeas Corpus n. 4026473-82.2018.8.24.0000\n\n...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,externo_controlador.php?acao=consultaArquivado...,Outro/Não identificado,Bovino (Gado/Vaca),São Bento do Sul
1270,4032299-55.2019.8.24.0000/TJSC,NaN,05/12/2019,PAULO HENRIQUE MORITZ MARTINS DA SILVA,NaN,NaN,NaN,Agravo de Instrumento n. 4032299-55.2019.8.24....,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,externo_controlador.php?acao=consultaArquivado...,Outro/Não identificado,Ave (Galo/Pássaro),None
1271,0006868-40.2013.8.24.0019/TJSC,NaN,19/05/2020,MONTEIRO ROCHA,NaN,NaN,NaN,Quinta Câmara de Direito Comercial\nDesembarga...,https://eproc2g.tjsc.jus.br/eproc/externo_cont...,externo_controlador.php?acao=consultaArquivado...,Outro/Não identificado,Ave (Galo/Pássaro),None


In [214]:
CAMINHO_ARQUIVO = os.path.join('data', 'maus-tratos_atualizado_classificado_comarca.csv')
df.to_csv(CAMINHO_ARQUIVO, index=False)